># AGENTE DE DIÁLOGO HÍBRIDO BASEADO EM REGRAS E DADOS
>CURSO: Tecnólogo em Inteligência Artificial Aplicada  
>DISCIPLINA: Agentes Conversacionais  
>AUTORA: Carla Edila Silveira  
>OBJETIVO: construir um agente de diálogo que trará ocorrências sobre determinado tema  
>MELHORIA:   
>GITHUB: https://github.com/rosacarla/Chatbot-baseado-em-regras-e-dados  
>DATA: 05/09/2023
______________________________________________________________________

<body>
<center>
<img src="https://i.postimg.cc/0Q7ZcBm7/header.png" align="middle">
</center>
</body>

>## 1. Qual o contexto do projeto?
><p align="justify">Um agente de diálogo de <i>question answering</i> que, baseado em um corpus de texto sobre um assunto, traz informações mais relevantes de acordo com a consulta do usuário.</p>

>## 2. Quais ferramentas e técnicas adotar?
>*   **NLTK** - toolkit de PLN em Python
>*   **Expressões Regulares** - pacote de regex do Python
>*   **urllib e BeautifulSoup** - bibliotecas para obter dados de páginas HTML
>*   **scikit-learn** - pacote com funcionalidades de manipulação de dados e Machine Learning (serão utilizados TF-IDF e Similaridade de cosseno)



> ## 3. Construção do agente de diálogo
>A operação do agente será deste modo:
>1.   Recebe **entrada** do usuário
>2.   **Pré-processa** a entrada do usuário
>3.   Calcula a **similaridade** entre a entrada e as sentenças do corpus
>4.   Obtém a sentença **mais similar do corpus**
>5.   Mostra-a como **resposta** ao usuário  

>Antes destas etapas, será criado o corpus ao obter automaticamente dados da Wikipedia.


> ## 4. Importação de bibliotecas
> Importar pacote de expressões regulares do Python e acesso ao WordNet dado pelo NLTK.

In [69]:
import nltk
nltk.download('punkt')
nltk.download('rslp')# Stemming em pt-br
from nltk.corpus import stopwords
nltk.download('stopwords')# Lista de stopwords
import numpy as np
import random
import string
import requests
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


>## 5. Construção do corpus
><p align="justify">Será feito um <i>web-scraping</i> para obter os dados automaticamente da Wikipedia. Este processo deve ser executado só uma vez, e o arquivo salvo em forma de texto na máquina.</p>

In [79]:
# Busca paginas sobre o BRASIL. Para mudar o tema basta colocar o link para outra pagina.
# Lista de URLs para as consultas
urls = [
    'https://pt.wikipedia.org/wiki/Brasil',
    'https://brasilescola.uol.com.br/geografia/pais-brasil.htm',
    'https://www.brasilparalelo.com.br/artigos/historia-do-brasil',
    'https://www.todamateria.com.br/a-historia-do-brasil/',
    'https://www.observatoriodasmetropoles.net.br/ibge-apresenta-os-primeiros-resultados-do-censo-2022/'
]

corpus = []  # Inicializa lista para armazenar o texto

for url in urls:
    try:
        # Faz solicitacao HTTP para obter conteudo da pagina
        codigo_html = requests.get(url)

        # Verifica se solicitacao foi bem-sucedida (código de status 200)
        if codigo_html.status_code == 200:
            # Processa o conteudo HTML da pagina
            html_processado = BeautifulSoup(codigo_html.text, 'html.parser')

            # Encontra todos os paragrafos do texto
            paragrafos = html_processado.find_all('p')

            # Concatena textos de paragrafos em unica string
            texto = '\n'.join([p.get_text() for p in paragrafos])

            # Normaliza texto para minusculas
            texto = texto.lower()

            # Adiciona texto ao corpus
            corpus.append(texto)

            # Imprime primeiros 1000 caracteres do texto
            print(f'Texto da página {url}:\n{texto[:1000]}\n')
        else:
            print(f'Falha ao acessar a página {url}: {codigo_html.status_code}')
    except Exception as e:
        print(f'Erro ao processar a página {url}: {str(e)}')

# Salva corpus em um arquivo de texto
with open('corpus.txt', 'w', encoding='utf-8') as arquivo:
    for texto in corpus:
        arquivo.write(texto + '\n')

Texto da página https://pt.wikipedia.org/wiki/Brasil:
coordenadas: 10° s 55° o
brasil (localmente [bɾaˈziw][b]), oficialmente república federativa do brasil (? escutar),[7] é o maior país da américa do sul e da região da américa latina, sendo o quinto maior do mundo em área territorial (equivalente a 47,3% do território sul-americano), com 8 510 417,771 km²,[1][8][9] e o sétimo em população[10][11] (com 203 milhões de habitantes, em agosto de 2022).[12] é o único país na américa onde se fala majoritariamente a língua portuguesa e o maior país lusófono do planeta,[13] além de ser uma das nações mais multiculturais e etnicamente diversas, em decorrência da forte imigração oriunda de variados locais do mundo. sua atual constituição, promulgada em 1988, concebe o brasil como uma república federativa presidencialista,[7] formada pela união dos 26 estados, do distrito federal e dos 5 570 municípios.[7][14][nota 1]

banhado pelo oceano atlântico, o brasil tem um litoral de 7 491 km[13] e faz 

>## 6. Pré-processamento do corpus
> É necessário remover caracteres especiais do texto e dividí-lo em sentenças válidas.

In [80]:
texto = re.sub(r'\[[0-9]*\]', ' ', texto)
texto = re.sub(r'\s+', ' ', texto)
texto[0:1000]

'de acordo com o instituto brasileiro de geografia e estatística (ibge), a população do brasil atingiu 203.062.512 pessoas, com um aumento de 12,3 milhões na última década – número abaixo da estimativa do órgão, que apontava para um total de 207,7 milhões de pessoas. os primeiros dados do censo demográfico 2022 foram divulgados na quarta-feira, dia 28, no auditório do museu do amanhã, localizado na cidade do rio de janeiro. a edição de 2022 começou a ser planejada em 2017, com a intenção de ser realizada em 2020. no entanto, devido à pandemia de covid-19, o processo foi interrompido e novos planos tiveram que ser traçados. ainda com incertezas, incluindo a questão orçamentária diante dos cortes no orçamento federal anunciados pelo governo bolsonaro, os preparativos só foram retomados em 2021. cimar azeredo pereira, presidente substituto do ibge, falou sobre os desafios enfrentados e mencionou a decisão de utilizar a ilha de paquetá (rj) como teste para o censo, uma vez que todos os hab

In [81]:
sentencas = nltk.sent_tokenize(texto, language='portuguese')
palavras = nltk.word_tokenize(texto, language='portuguese')
sentencas[10:15]

['os dispositivos móveis de coleta (dmcs) utilizaram sistemas georreferenciados, chips 4g e wifi, o que tornou a gestão da coleta de dados mais ágil.',
 'agora, esses dispositivos serão utilizados pelas secretarias de saúde em todo o brasil.',
 'os dados preliminares mostram que a população do brasil atingiu 203.062.512 pessoas.',
 'o censo 2022 registrou um aumento de 12,3 milhões desde 2010, o que representa um crescimento médio da população de 0,52% nos últimos anos.',
 'o país agora tem 90,7 milhões de domicílios, 34% a mais do que há uma década.']

>## 7. Funções de pré-processamento de entrada do usuário
><p align="justify">Cria funções para pré-processar as entradas do usuário, com retirada de  pontuações e uso de Stemming nos textos, para que palavras similares sejam processadas igualmente pelo algoritmo (por ex., pedra e pedregulho teriam mesma forma léxica).</p>

In [82]:
# Define funcao que faz Stemming em todo texto
def stemming(tokens):
  stemmer = nltk.stem.RSLPStemmer()
  novotexto = []
  for token in tokens:
    novotexto.append(stemmer.stem(token.lower()))
  return novotexto

# Funcao que remove pontuacao, stopwords e aplica stemming
def preprocessa(documento):
  # Remove pontuacao
  documento = documento.translate(str.maketrans('', '', string.punctuation))

  # Tokenização de palavras
  tokens = nltk.word_tokenize(documento, language='portuguese')

  # Remove stopwords
  stopwords_pt = set(stopwords.words('portuguese'))
  tokens = [token.lower() for token in tokens if token.lower() not in stopwords_pt]
  removePontuacao = dict((ord(punctuation), None) for punctuation in string.punctuation)

  # Aplica stemming
  tokens = stemming(tokens)

  return tokens

In [83]:
# Conferir como fica um texto apos seu pre processamento
preprocessa("Olá meu nome é Lucas, eu moro no Brasil, e você?")

['olá', 'nom', 'luc', 'mor', 'brasil']

>## 8. Resposta a saudações
><p align="justify">Embora seja um sistema de diálogo baseado em tarefas, é provável que o usuário inicie conversas com saudações ao agente. Por isso, será desenvolvida uma função (regras) para tratar desta situação.</p>  
><p align="justify">Serão criadas algumas respostas possíveis, dentre as quais serão escolhidas algumas aleatoriamente, para evitar que o agente fique repetitivo.</p>

In [84]:
import random

saudacoes_entrada = ("olá", "bom dia", "boa tarde", "boa noite", "oi", "como vai", "e aí", "tudo bem")
saudacoes_respostas = ["olá", "olá, espero que esteja tudo bem contigo", "Olá! Como posso ajudar você hoje?", "oi", "Oie",
                       "Seja bem-vindo, em que posso te ajudar?", "Oi! Estou à disposição para responder às suas perguntas.",
                       "Olá! Estou aqui para fornecer informações sobre o Brasil. O que você gostaria de saber?",
                       "Oi! Bem-vindo! O que você deseja saber sobre o Brasil?"]

# Funcao para responder a saudacao
def geradorsaudacoes(saudacao):
    for token in saudacao.split():
        if token.lower() in saudacoes_entrada:
            return random.choice(saudacoes_respostas)

    # Se nenhuma saudacao conhecida for encontrada, retorna None
    return None

In [85]:
# Ao executar este exemplo várias vezes serao vistas respostas diferentes
geradorsaudacoes('Olá')

'Oi! Bem-vindo! O que você deseja saber sobre o Brasil?'

>## 9. Resposta a consultas do usuário
><p align="justify">Função para tratar consultas do usuário, com a comparação da similaridade entre a entrada do usuário e as sentenças do corpus. Se houver, a sentença mais similar será mostrada como resposta.</p>
><p align="justify">Nesta função utilizam-se os algoritmos TD-IDF (<i>Term Frequency – Inverse Document Frequency</i>) e similaridade de cosseno.</p>

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [87]:
# Funcao para responder a consultas do usuario
def geradorrespostas(entradausuario, corpus):
  resposta = ''

  # Verifica se entrada do usuario eh string
  if isinstance(entradausuario, str):

   # Pre rocessamento da entrada do usuario
    entrada_preprocessada = preprocessa(entradausuario)

    # Pre processamento do corpus por sentencas
    corpus_preprocessado = [preprocessa(sentenca) for sentenca in corpus]

    # Cria vetorizador TF-IDF
    vectorizer = TfidfVectorizer(tokenizer=preprocessa, stop_words=stopwords.words('portuguese'))

    # Calcula os vetores TF-IDF para o corpus
    tfidf_corpus = vectorizer.fit_transform(corpus_preprocessado)

    # Calcula similaridade de cosseno entre vetor da entrada do usuário e vetores TF-IDF do corpus
    similaridades = cosine_similarity(vectorizer.transform([entrada_preprocessada]), tfidf_corpus)

    # Encontra indice da sentenca mais similar
    indice_mais_similar = np.argmax(similaridades)

    # Calcula similaridade maxima
    similaridade_maxima = similaridades[0, indice_mais_similar]

    if similaridade_maxima == 0:
      resposta = resposta + "Me desculpe, não entendi o que você pediu."
    else:
      resposta = resposta + corpus[indice_mais_similar]

    return resposta

  #sentencas.append(entradausuario)

  #word_vectorizer = TfidfVectorizer(tokenizer=preprocessa, stop_words=stopwords.words('portuguese'))
  #all_word_vectors = word_vectorizer.fit_transform(sentencas)
  #similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
  #similar_sentence_number = similar_vector_values.argsort()[0][-2]

  #matched_vector = similar_vector_values.flatten()
  #matched_vector.sort()
  #vector_matched = matched_vector[-2]

  #if vector_matched == 0:
    #resposta = resposta + "Me desculpe, não entendi o que você pediu."
    #return resposta
  #else:
    #resposta = resposta + sentencas[similar_sentence_number]
    #return resposta

>## 10. Interação com o agente de diálogo
><p align="justify">Define um algoritmo que continue interagindo com o usuário até que ele decida finalizar.</p>  
><p align="justify">O resultado não é sempre o ideal, porém cobre muitas das possíveis perguntas. Se utilizássemos apenas regras de diálogo para responder perguntas sobre um assunto, precisaríamos de centenas de regras.</p>
><p align="justify"> Como as respostas são baseadas em dados, apenas uma regra que calcula similaridade com o corpus é suficiente. Faça perguntas como:</p>  

>*  *Qual o esporte mais popular no Brasil?*
>*  *Quais eventos esportivos o Brasil já organizou?*
>*  *Como é a cozinha brasileira?*
>*  *Onde são realizadas pesquisas tecnológicas no Brasil?*


In [88]:
# Inicia interacao com usuario
print("Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre nosso país ou diga 'tchau' para sair.")
continue_dialogue = True
while continue_dialogue:
  # Obtem entrada do usuario
  human_text = input().lower()

  if human_text != 'tchau':
    if human_text in ['obrigado', 'muito obrigado', 'agradecido', 'valeu', 'agradeço', 'beleza']:
      continue_dialogue = False
      print("Agente Tupiniquim: Disponha!")
    else:
      if geradorsaudacoes(human_text) != None:
        print("Agente Tupiniquim: " + geradorsaudacoes(human_text))
      else:
        resposta = geradorrespostas(human_text, corpus)
        if resposta:
          print("Agente Tupiniquim: ", resposta)
        else:
          print("Agente Tupiniquim: Me desculpe, não entendi o que você pediu.")
  else:
    continue_dialogue = False
    print("Agente Tupiniquim: Até a próxima!")

Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre nosso país ou diga 'tchau' para sair.
oi
Agente Tupiniquim: Oi! Bem-vindo! O que você deseja saber sobre o Brasil?
qual o esporte mais popular no Brasil?


AttributeError: ignored

>## 11. Como melhorar o projeto?
><p align="justify">Este agente de diálogo utiliza modelo baseado em regras, em que uma das regras usa corpus de dados para formular respostas. Desse modo, o modelo ficou mais flexível, sem necessidade de criar centenas/milhares de regras. Vejamos como melhorar o projeto:</p>  
><p align="justify">A) Além dos parágrafos (tag "p") da página da Wikipedia, podem ser usados dados dispostos na coluna direita, que trazem informações relevantes, como população, atual presidente etc., para montar sentenças.</p>  
><p align="justify">B) Melhorar o cálculo de similaridade com uso de um modelo de Word Embeddings, além do TF-IDF.</p>
><p align="justify">C) Obter dados sobre o Brasil a partir de diferentes fontes.</p>
><p align="justify">D) Criar classificador de contexto para o agente e, de modo dinâmico, buscar páginas da Wikipedia correspondentes à pergunta do usuário, para depois dar a resposta. Desse modo o agente não se limitaria a perguntas sobre o Brasil.</p>



>## 12. Referências e material complementar  
>* [Python for NLP: Creating a Rule-Based Chatbot](https://stackabuse.com/python-for-nlp-creating-a-rule-based-chatbot/)
>* [Building a Simple Chatbot from Scratch in Python (using NLTK)](https://morioh.com/p/6cc33336784c)
>* [Building a simple chatbot in python](https://medium.com/nxtplus/building-a-simple-chatbot-in-python-3963618c490a)
>* [Designing A ChatBot Using Python: A Modified Approach](https://towardsdatascience.com/designing-a-chatbot-using-python-a-modified-approach-96f09fd89c6d)
>* [Build Your First Python Chatbot Project](https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro)
>* [Python Chatbot Project – Learn to build your first chatbot using NLTK & Keras](https://data-flair.training/blogs/python-chatbot-project/)
>* [Python Chat Bot Tutorial - Chatbot with Deep Learning (Part 1)](https://www.youtube.com/watch?v=wypVcNIH6D4)
>* [Intelligent AI Chatbot in Python](https://www.youtube.com/watch?v=1lwddP0KUEg)
>* [Coding a Jarvis AI Using Python 3 For Beginners](https://www.youtube.com/watch?v=NZMTWBpLUa4)  
>* Projeto elaborado a partir de Notebook criado por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).